In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [2]:
from my_models.spvd import SPVUnet

down_blocks = [{
    "features_list": (32, 64, 128, 192, 192),
    "num_layers_list": 1,
    "attn_heads": None
}]
up_blocks = [{
    "features_list": (32, 64, 128, 192, 192),
    "num_layers_list": 1,
    "attn_heads": None
}]
t_emb_features = 64

s1 = SPVUnet(down_blocks, up_blocks, t_emb_features)

import models

lr = 1e-4
s1 = models.DiffusionBase(s1, lr=lr)

/opt/conda/envs/spvd/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
from datasets.modelnet40.modelnet40_loader import get_dataloaders, ModelNet40, ModelNet40Sparse
path = "../data/ModelNet40"
categories = ["xbox"]
tr, te = get_dataloaders(path, categories=categories)

v = next(iter(te))
x, t, f = v['input'], v['t'], v['render-features']

Loading renders for xbox: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:02<00:00,  7.71it/s]


In [4]:
import lightning as L
trainer = L.Trainer(
    max_epochs=1, 
    gradient_clip_val=10.0,
)

trainer.fit(model=s1, train_dataloaders=tr, val_dataloaders=te)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA A10G') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type    | Params | Mode 
------------------------------------------
0 | model | SPVUnet | 9.5 M  | train
------------------------------------------
9.5 M     Trainable params
0         Non-trainable params
9.5 M     Total params
38.141    Total estimated model params size (MB)
149       Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                                                                            …

/opt/conda/envs/spvd/lib/python3.10/site-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (4) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |                                                                                                   …

Validation: |                                                                                                 …

`Trainer.fit` stopped: `max_epochs=1` reached.


In [ ]:
import models

m = models.SPVD()

lr = 1e-4
model = models.DiffusionBase(m, lr=lr)

import lightning as L
trainer = L.Trainer(
    max_epochs=1, 
    gradient_clip_val=10.0,
)

trainer.fit(model=model, train_dataloaders=tr, val_dataloaders=te)